In [200]:
import tensorflow
from tensorflow.keras.layers import Layer,Dense,Embedding,Lambda,Multiply,Input,Flatten,Concatenate
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

In [201]:
import tensorflow as tf
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load the Boston Housing Prices dataset
boston = load_diabetes()
X, y = boston.data, boston.target

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Layer And Model

In [202]:
class SuperDense(Layer):
    
    def __init__(self, *args, **kwargs):
        super(SuperDense, self).__init__()
        
        self.dense2 = Dense(128, activation="softmax")
        self.lamb = Lambda(self.softLambda)
        self.embed = Embedding(input_dim=128, output_dim=32)
        self.var = [i for i in range(128)]
        self.my_variable = self.add_weight(
            name="my_variable",
            shape=(1,),
            initializer="zeros",
            trainable=True,
        )

    def softLambda(self, x):
        return K.cast(K.greater_equal(x, 0.5), K.floatx())

    def call(self, inputs):
        x = self.dense2(inputs)
        x = self.lamb(x)
        
        # Ensure x has the shape [batch_size, 128]
        x = Multiply()([K.constant(self.var,shape=(1, 128)), x])
        x = self.embed(x)
        x = Flatten()(x)        
        return x


In [203]:
ip=Input((X_scaled.shape[1],))
y=Dense(124,activation="relu")(ip)
z=Dense(128,activation="relu")(y)
y=SuperDense()(y)
cnc=Concatenate()([z,y])
dense2=Dense(12,activation="relu")(cnc)
op=Dense(1,activation="relu")(dense2)

In [204]:
model=Model([ip],[op])

In [205]:
model.compile(optimizer="adam",loss="mae")

In [206]:
ip2=Input((X_scaled.shape[1],))
y2=Dense(124,activation="relu")(ip2)
z2=Dense(128,activation="relu")(y2)
y2=Dense(128,activation="relu")(y2)
cnc2=Concatenate()([z2,y2])
dense22=Dense(256,activation="relu")(cnc2)
op2=Dense(1,activation="relu")(dense22)

In [207]:
model2=Model([ip2],[op2])

In [208]:
model2.compile(optimizer="adam",loss="mae")

# **Regression Test**

In [213]:
# Train the model
model.fit(X_train,y_train, epochs=200,validation_split=0.3)

Epoch 1/200
8/8 [==============================] - 0s 13ms/step - loss: 29.4908 - val_loss: 47.9161
Epoch 2/200
8/8 [==============================] - 0s 8ms/step - loss: 29.6434 - val_loss: 47.1264
Epoch 3/200
8/8 [==============================] - 0s 8ms/step - loss: 29.3068 - val_loss: 48.0511
Epoch 4/200
8/8 [==============================] - 0s 8ms/step - loss: 29.1175 - val_loss: 47.1197
Epoch 5/200
8/8 [==============================] - 0s 8ms/step - loss: 29.2195 - val_loss: 47.9124
Epoch 6/200
8/8 [==============================] - 0s 8ms/step - loss: 28.9475 - val_loss: 47.1534
Epoch 7/200
8/8 [==============================] - 0s 8ms/step - loss: 28.8967 - val_loss: 48.0047
Epoch 8/200
8/8 [==============================] - 0s 8ms/step - loss: 28.8998 - val_loss: 47.3630
Epoch 9/200
8/8 [==============================] - 0s 8ms/step - loss: 28.8974 - val_loss: 48.0731
Epoch 10/200
8/8 [==============================] - 0s 8ms/step - loss: 28.7510 - val_loss: 47.4543
Epoch 11

In [214]:
model2.fit(X_train,y_train,epochs=200,validation_split=0.3)

Epoch 1/200
8/8 [==============================] - 0s 12ms/step - loss: 12.3905 - val_loss: 54.6211
Epoch 2/200
8/8 [==============================] - 0s 7ms/step - loss: 12.3370 - val_loss: 55.9137
Epoch 3/200
8/8 [==============================] - 0s 7ms/step - loss: 11.2096 - val_loss: 55.8634
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 11.1119 - val_loss: 56.4323
Epoch 5/200
8/8 [==============================] - 0s 7ms/step - loss: 11.5393 - val_loss: 56.1552
Epoch 6/200
8/8 [==============================] - 0s 8ms/step - loss: 11.4048 - val_loss: 55.9647
Epoch 7/200
8/8 [==============================] - 0s 9ms/step - loss: 12.6079 - val_loss: 56.2876
Epoch 8/200
8/8 [==============================] - 0s 7ms/step - loss: 11.0317 - val_loss: 56.1927
Epoch 9/200
8/8 [==============================] - 0s 7ms/step - loss: 11.9142 - val_loss: 55.4992
Epoch 10/200
8/8 [==============================] - 0s 7ms/step - loss: 12.5017 - val_loss: 56.3688
Epoch 11

In [211]:
model.summary()

Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_33 (InputLayer)       [(None, 10)]                 0         []                            
                                                                                                  
 dense_168 (Dense)           (None, 124)                  1364      ['input_33[0][0]']            
                                                                                                  
 dense_169 (Dense)           (None, 128)                  16000     ['dense_168[0][0]']           
                                                                                                  
 super_dense_15 (SuperDense  (None, 4096)                 20097     ['dense_168[0][0]']           
 )                                                                                         

In [212]:
model2.summary()

Model: "model_35"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_34 (InputLayer)       [(None, 10)]                 0         []                            
                                                                                                  
 dense_173 (Dense)           (None, 124)                  1364      ['input_34[0][0]']            
                                                                                                  
 dense_174 (Dense)           (None, 128)                  16000     ['dense_173[0][0]']           
                                                                                                  
 dense_175 (Dense)           (None, 128)                  16000     ['dense_173[0][0]']           
                                                                                           